# Demoing fetching PV locations from DuckLake into a LanceDB dataset



### References:
- [DuckLake Documentation](https://ducklake.select/docs/stable/)
- [DuckLake with Ibis Python DataFrames](https://emilsadek.com/blog/ducklake-ibis/)
- [A new data lakehouse with DuckLake and dbt](https://giacomo.coletto.io/blog/ducklake/)

In [1]:
import ibis
from ibis import _
import ibis.selectors as s
import duckdb
import pandas as pd
from huggingface_hub import HfFileSystem, login

import geopandas as gpd
import shapely
from shapely import wkt
import s2cell

from tqdm import tqdm
from dotenv import load_dotenv

import os
import shutil
import random
import pickle
from pathlib import Path
from collections import Counter

In [2]:
ibis.options.interactive = True
ibis.options.graphviz_repr=True
random.seed(24765131)

load_dotenv()

# assume we're using prod catalog, but default to local/dev if env var not set
local_default = os.getenv('DUCKLAKE_CONNECTION_STRING_DEV')
DUCKLAKE_CATALOG = os.getenv('DUCKLAKE_CONNECTION_STRING_PROD', local_default)
DUCKLAKE_ATTACH = os.getenv("DUCKLAKE_ATTACH_PROD")
DUCKLAKE_NAME = os.getenv("DUCKLAKE_NAME")
DUCKLAKE_DATA_PATH = os.getenv("DUCKLAKE_DATA_PATH")

# pretty print our connection string info 
# TODO: comment out and remove output before commit
print(f"Using DuckLake catalog type: {DUCKLAKE_CATALOG.split(':')[1]}" )
catalog_creds = DUCKLAKE_CATALOG.split(':')[2].strip('()').split(' ')
# skip DATA_PATH at end
for cred in catalog_creds[:-2]:
    key, val = cred.split('=')
    # print(f"  {key}: {val}")
print(f"  DATA_PATH: {DUCKLAKE_CATALOG.split('DATA_PATH ')[1][:-1]}")

Using DuckLake catalog type: postgres
  DATA_PATH: 's3://eo-pv-lakehouse/ducklake_data'


### Connect to our data lake catalog with ibis 



In [3]:
duckdb_config = {
    'threads': 6,
    'memory_limit': '12GB',
    's3_access_key_id': os.getenv('R2_ACCESS_KEY_ID'),
    's3_secret_access_key': os.getenv('R2_SECRET_KEY'),
    's3_endpoint': os.getenv('R2_S3_ENDPOINT'),
    's3_use_ssl': 'true',
    's3_url_style': 'path'
}

# use in-memory/ephemeral db
con = ibis.duckdb.connect(
    extensions=["ducklake", "spatial", "h3", "httpfs"],
    **duckdb_config
    )

# r2_bucket_setup = f"""SET s3_access_key_id='{os.getenv('R2_ACCESS_KEY_ID')}';
# SET s3_secret_access_key='{os.getenv('R2_SECRET_KEY')}';
# SET s3_endpoint='{os.getenv('R2_S3_ENDPOINT')}';
# SET s3_use_ssl='true';
# SET s3_url_style='path';
# """
# con.raw_sql(r2_bucket_setup)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# con.load_extension("ducklake")
attach_catalog_sql = f"""ATTACH IF NOT EXISTS '{DUCKLAKE_ATTACH}' AS {DUCKLAKE_NAME}
    (DATA_PATH '{DUCKLAKE_DATA_PATH}');
USE {DUCKLAKE_NAME};
"""
con.raw_sql(attach_catalog_sql)
# add community cache_httpfs extension; this causes an "http_init already loaded" error only when loading as part of ibis extensions arg
# fails in Windows; commented out by default
# con.raw_sql("INSTALL cache_httpfs FROM community; LOAD cache_httpfs;")
con.list_catalogs()

['__ducklake_metadata_eo_pv_lakehouse',
 'eo_pv_lakehouse',
 'memory',
 'system',
 'temp']

In [10]:
from ibis import _ 

# ibis loading of extensions fails in colab and Windows
con.raw_sql("INSTALL h3 FROM community; LOAD h3;")
# test h3 functionality: https://github.com/isaacbrodsky/h3-duckdb?tab=readme-ov-file#full-list-of-functions
@ibis.udf.scalar.builtin
def h3_latlng_to_cell_string(lat: float, lng: float, resolution: int) -> str:
    '''Convert latitude/longitude coordinate to cell ID'''

# and ibis udf that enables using backend's (duckdb) functions including extensions: https://ibis-project.org/reference/scalar-udfs#ibis.expr.operations.udf.scalar.builtin
h3_test = stg_pv.sample(0.01).select("unified_id", "centroid_lat", "centroid_lon").\
    mutate(h3_cell=h3_latlng_to_cell_string(_.centroid_lat, _.centroid_lon, 8)).head(10)
display(h3_test)
con.table("stg_pv_consolidated").sample(0.01).select("unified_id", "centroid_lat", "centroid_lon").\
    mutate(h3_cell=h3_latlng_to_cell_string(_.centroid_lat, _.centroid_lon, 8)).head(10).visualize(format='png')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

TypeError: 'pyarrow.lib.RecordBatchReader' object is not subscriptable

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [6]:
con.list_tables()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

['pv_h3_cells',
 'pv_h3_grid',
 'raw_chn_med_res_pv_2024',
 'raw_global_harmonized_large_solar_farms_2020',
 'raw_global_pv_inventory_sent2_spot_2021',
 'raw_ind_pv_solar_farms_2022',
 'raw_uk_crowdsourced_pv_2020',
 'raw_usa_cali_usgs_pv_2016',
 'stg_chn_med_res_pv_2024',
 'stg_global_harmonized_large_solar_farms_2020',
 'stg_global_pv_inventory_sent2_spot_2021',
 'stg_ind_pv_solar_farms_2022',
 'stg_pv_consolidated',
 'stg_uk_crowdsourced_pv_2020',
 'stg_usa_cali_usgs_pv_2016']

In [9]:
stg_pv = con.table("stg_pv_consolidated")
# exclude the uk dataset as it seems to have no intersects and severely distorts our matching % 
stg_pv = stg_pv.filter(_.dataset_name != 'uk_crowdsourced_pv_2020')

@ibis.udf.scalar.builtin
# signature returns GEOMETRY; use ibis geometry datatype
def ST_GeomFromText(wkt: str) -> ibis.expr.datatypes.GeoSpatial:
    '''Convert WKT to geometry'''
stg_pv = stg_pv.mutate(geom=ST_GeomFromText(_.geometry))

# sample 
stg_pv.sample(0.01)

TypeError: 'pyarrow.lib.RecordBatchReader' object is not subscriptable

In [ ]:

stg_pv.aggregate(by=["h3_index_8"], pv_count=_.unified_id.count(), pv_area=_.area_m2.sum()).order_by(ibis.desc("pv_count")).head(20)

In [ ]:
# display(stg_pv.aggregate(by=["h3_index_8"], pv_count=_.unified_id.count(), pv_area=_.area_m2.sum()).order_by(ibis.desc("pv_count")).head(20).visualize())

In [ ]:
# load h3 duckdb extension and calculate coarser h3 index
con.load_extension("h3")
# test returning column of h3 cells 
stg_pv.sql("SELECT h3_cell_to_parent(h3_index_8, 6) FROM stg_pv_consolidated LIMIT 10").head()

In [ ]:
# Querying (remote) Hub files with DuckDB

fs = HfFileSystem()
duckdb.register_filesystem(fs)
# Query a remote file and get the result back as a dataframe
# fs_query_file = "hf://datasets/my-username/my-dataset-repo/data_dir/data.parquet"
# df = duckdb.query(f"SELECT * FROM '{fs_query_file}' LIMIT 10").df()

## MVP Workflow: S2 Cell Matching with core-five Dataset

This section implements the barebones workflow:
1. Convert PV labels from DuckLake to GeoPandas
2. Parse S2 cell IDs from core-five HuggingFace dataset structure
3. Add S2 cell columns to PV dataframe at appropriate levels
4. Match PV S2 cells with core-five S2 cells
5. Set up basic xarray retrieval with virtualizarr optimization

### Step 1: Convert PV Labels to GeoPandas DataFrame

In [ ]:
# Fetched PV labels from DuckLake using ibis
# Convert to pandas first; shuffle the rows (revise as dataset grows)
pv_df = stg_pv.to_pandas().sample(frac=1)

# Select a sample for MVP (e.g., 150-200 labels)
# pv_sample = pv_df.sample(n=200, random_state=42)
pv_sample = pv_df 

# Create GeoPandas dataframe and convert WKT geometry strings to shapely geometries
pv_gdf = gpd.GeoDataFrame(pv_sample, geometry=pv_sample['geometry'].apply(shapely.wkt.loads), crs='EPSG:4326')

print(f"Loaded {len(pv_gdf)} PV labels into GeoPandas")
print(f"Columns: {pv_gdf.columns.tolist()}")
pv_gdf.head()

## Hugging Face Hub

The [Hugging Face Hub](https://huggingface.co/docs/hub/index) is a Machine Learning platform with hundreds of thousands of open source and publicly available datasets, models, and interactive model demos. 

### Step 2: Parse S2 Cell IDs from core-five HuggingFace Dataset

In [ ]:
# OPTIMIZED: Use fs.glob() to get all files in ONE API call (avoids rate limiting)
# Also parse BOTH parent dirs (level 10) AND child files (level 13) for hierarchical matching

cache_file = Path("../data/core_five_s2_mapping.pkl")
core_five_root = "datasets/gajeshladhar/core-five/src/datatree"
all_nc_files = None

if cache_file.exists() and cache_file.stat().st_size > 0:
    print(f"Loading from cache: {cache_file}")
    with open(cache_file, 'rb') as f:
        s2_cell_mapping = pickle.load(f)
    parent_cells_seen = set([info['token'] for info in s2_cell_mapping.values() if info['is_parent']])
    print(f"  Found {len(parent_cells_seen)} parent cells from cache")
    all_nc_files = [info['file_path'] for info in s2_cell_mapping.values() if not info['is_parent']]
else:
    print("Fetching all .nc files from core-five (single API call)...")
    all_nc_files = fs.glob(f"{core_five_root}/**/*.nc")  # ONE call gets all ~93k files
    print(f"Found {len(all_nc_files)} .nc files")
    
    s2_cell_mapping = {}
    parent_cells_seen = set()
    
    for file_path in tqdm(list(all_nc_files), desc="Parsing S2 cells"):
        parts = Path(file_path).parts
        parent_token = parts[-2]  # e.g., "1a220b"
        child_token = Path(file_path).stem  # e.g., "1a220c04"
        
        # Parse parent cell (only once per unique parent)
        if parent_token not in parent_cells_seen:
            try:
                parent_cell_id = s2cell.token_to_cell_id(parent_token)
                s2_cell_mapping[parent_cell_id] = {
                    'token': parent_token,
                    'level': s2cell.token_to_level(parent_token),
                    'file_path': str(Path(file_path).parent),
                    'is_parent': True,
                    'children': []
                }
                parent_cells_seen.add(parent_token)
            except:
                print(f"Failed to parse parent cell: {parent_token}")
                pass
        
        # Parse child cell
        try:
            child_cell_id = s2cell.token_to_cell_id(child_token)
            parent_cell_id = s2cell.token_to_cell_id(parent_token)
            
            s2_cell_mapping[child_cell_id] = {
                'token': child_token,
                'level': s2cell.cell_id_to_level(child_cell_id),
                'file_path': file_path,
                'is_parent': False,
                'parent_cell_id': parent_cell_id
            }
            
            # Link child to parent
            if parent_cell_id in s2_cell_mapping:
                s2_cell_mapping[parent_cell_id]['children'].append(child_cell_id)
        except:
            continue
    
    # Save cache if successful
    if s2_cell_mapping:
        cache_file.parent.mkdir(parents=True, exist_ok=True)
        with open(cache_file, 'wb') as f:
            pickle.dump(s2_cell_mapping, f)
        print(f"Saved to cache: {cache_file}")

# Analyze levels; we only expect 2 levels: one for parent_dir and a consistent s2 cell level for child dirs
levels = sorted(list(set([info['level'] for info in s2_cell_mapping.values()]))) # sort so we start with parent dir level
print(f"\nParsed {len(s2_cell_mapping)} total S2 cells")
print(f"  Parent cells: {len(parent_cells_seen) if 'parent_cells_seen' in locals() else 'N/A'}")
print(f"\nS2 levels in core-five:")
for level in levels:
    if level == min(levels):
        print(f"  Parent dir level: {level}; number of cells: {len(parent_cells_seen)}")
    else:
        print(f"  Child file level: {level}; number of cells: {len([info for info in s2_cell_mapping.values() if info['level'] == level])}")

## Google S2 Geometric Spatial Indexing

See python S2 library: [S2Cell Docs](https://docs.s2cell.aliddell.com/en/stable/index.html)

See details: https://learn.microsoft.com/en-us/kusto/query/geo-point-to-s2cell-function?view=microsoft-fabric

### Step 3: Add S2 Cell Columns to PV DataFrame

In [ ]:
# Determine which S2 levels to use based on core-five dataset
# We'll add columns for each level present in core-five

print(f"Adding S2 cell columns for levels: {levels}")

# For each PV label, compute S2 cell IDs at each target level using centroid
for level in levels:
    col_name = f's2_cell_lvl_{level}'
    pv_gdf[col_name] = pv_gdf.apply(
        lambda row: s2cell.lat_lon_to_cell_id(
            row['centroid_lat'], 
            row['centroid_lon'], 
            level
        ),
        axis=1
    )
    print(f"  Added column: {col_name}")

print(f"\nPV GeoDataFrame now has {len(pv_gdf.columns)} columns")
print(f"New S2 columns: {[col for col in pv_gdf.columns if col.startswith('s2_cell_lvl')]}")
pv_gdf.head()

### Step 4: Match PV S2 Cells with core-five S2 Cells

In [ ]:
# CHILD-ONLY MATCHING: Only match exact child cells (level 13)
# Parent matching would include ALL children (false positives)
# We only want the specific child cell where the PV label is located

def find_matches_child_only(row, s2_mapping):
    """Match only at child level for precise location"""
    matches = set()
    
    # Only match exact child cell (level 13)
    if 's2_cell_lvl_13' in row:
        child_id = row['s2_cell_lvl_13']
        if child_id in s2_mapping and not s2_mapping[child_id].get('is_parent', False):
            matches.add(s2_mapping[child_id]['file_path'])
    
    # NOTE: We still record parent cell IDs in the dataframe for reference,
    # but don't use them for matching to avoid false positives
    
    return list(matches)

pv_gdf['matched_files'] = pv_gdf.apply(lambda row: find_matches_child_only(row, s2_cell_mapping), axis=1)
pv_gdf['num_matches'] = pv_gdf['matched_files'].apply(len)

# Statistics
total = len(pv_gdf)
with_matches = (pv_gdf['num_matches'] > 0).sum()
print(f"\nMatching Results:")
print(f"  Total PV labels: {total:,}")
print(f"  Labels with matches: {with_matches:,} ({with_matches/total*100:.2f}%)")
print(f"  Labels without matches: {total - with_matches:,}")
print(f"  Average matches per label: {pv_gdf['num_matches'].mean():.2f}")

pv_gdf[pv_gdf['num_matches'] > 0][['unified_id', 'centroid_lat', 'centroid_lon', 'num_matches']].head()

### Step 4.5: Add Granular S2 Cells for Precise PV Label Capture

Based on S2 Cell statistics (http://s2geometry.io/resources/s2cell_statistics):
- **Level 13** (current): ~850m - 1185m edge length, ~0.76 - 1.59 km² area
- **Level 17**: ~53m - 77m edge length, ~2970 - 6227 m² area ✅ **RECOMMENDED** 
- **Level 18**: ~27m - 38m edge length, ~742 - 1556 m² area 
- **Level 19**: ~13m - 19m edge length, ~185 - 389 m² area 
<!-- - **Level 20**: ~7m - 10m edge length, ~46 - 97 m² area -->

Level 19 provides ~15m resolution, ideal for capturing individual PV installations.

In [ ]:
# Add level 19 S2 cells for precise PV label capture
S2_CELL_LVL = 17
def add_granular_s2_cell(row, level=19):
    """
    Add a granular S2 cell ID at specified level for precise spatial indexing.

    Args:
        row: DataFrame row with 'centroid' geometry
        level: S2 cell level (default 19 for ~15m resolution)
    """
    cell_id = s2cell.lat_lon_to_cell_id(row['centroid_lat'], row['centroid_lon'], level)
    return cell_id

# Add level 18 cells to our PV dataframe (better size for image chips)
print(f"Adding level {S2_CELL_LVL} S2 cells for PV label capture...")
pv_gdf[f's2_cell_lvl_{S2_CELL_LVL}'] = pv_gdf.apply(lambda row: add_granular_s2_cell(row, level=17), axis=1)

print(f"\nS2 Cell levels now available:")
print(f"  - Level 10 (parent): ~7-10 km edge, for directory matching")
print(f"  - Level 13 (child): ~850m-1185m edge, for file matching")
print(f"  - Level 17 (granular): ~53m - 77m edge length, for image chip-sized PV capture")

# Show sample
print(f"\nSample PV labels with all S2 levels:")
# NOTE: we have several labels with NULL unified_id (must be from a null field in our coalesce or hash processing during stg)
display(pv_gdf[['unified_id', 's2_cell_lvl_10', 's2_cell_lvl_13', f's2_cell_lvl_{S2_CELL_LVL}']].sample(5))
# get count of matching labels grouped by dataset
pv_gdf[pv_gdf['num_matches'] > 0].groupby('dataset_name')['num_matches'].sum()

### Step 5: Basic xarray Retrieval with VirtualiZarr Groundwork

In [ ]:
import xarray as xr
from obstore.store import from_url
from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

# Set up object store for HuggingFace dataset access
# Note: HuggingFace uses HTTPS, so we'll use HTTP store
bucket = "https://huggingface.co"
store = from_url(bucket)
registry = ObjectStoreRegistry({bucket: store})

print("Object store registry initialized for HuggingFace")

In [ ]:
# OPTION 1: In-memory loading with fsspec + h5netcdf (no local files)
# This opens the file via HTTP and reads into memory without saving to disk

import fsspec

SAMPLE_CRS = None

num_row_matches = (pv_gdf['num_matches'] > 0).sum()
rand_sample_idx = random.randint(0, num_row_matches - 1)
if num_row_matches > 0:
    sample_file = pv_gdf[pv_gdf['num_matches'] > 0].iloc[rand_sample_idx]['matched_files'][0]
    
    # Construct HuggingFace resolve URL
    filename = "/".join(Path(sample_file).parts[3:])  # Remove 'datasets/gajeshladhar/core-five/'
    url = f"https://huggingface.co/datasets/gajeshladhar/core-five/resolve/main/{filename}"
    
    print(f"Loading from URL: {url}")
    
    # Open file via HTTP using fsspec (in-memory)
    with fsspec.open(url, 'rb') as f:
        # Use h5netcdf engine (supports file-like objects)
        tree = xr.open_datatree(f, engine='h5netcdf')
        tree = tree.load()  # Load data into memory
        
        print(f"\nDataTree structure:")
        print(tree)
        
        # Access modalities
        if "hr/data" in tree:
            hr_data = tree["hr/data"]
            print(f"\nHigh-res RGB: {hr_data.dims}")
        
        if "s2" in tree:
            s2_data = tree["s2"]
            print(f"Sentinel-2: {s2_data.dims}")

        # CRS varies by S2 Cell image but is consistent within each .nc file between the modalities
        SAMPLE_CRS = tree.crs

else:
    print("No matches found.")

In [ ]:
# compare matching gdf with s2 cells vs direct pv label intersections with the tree.polygon geom
# see ibis geospatial functions here: https://ibis-project.org/reference/expression-geospatial
stg_pv.filter(stg_pv.geom.intersects(tree.polygon)).to_pandas().shape[0]
# note that the lower number of matches here is expected as this is only matching with a single image

In [ ]:
# test collecting the polygons of multiple images in a list (requires loading entire .nc file! run for performance comparison)
core_five_samples = random.sample(all_nc_files, 3) if 'all_nc_files' in locals() else random.sample(fs.glob(f"{core_five_root}/**/*.nc"), 3)
sample_polygons = []
for sample in tqdm(core_five_samples, desc="Collecting polygons"):
    filename = "/".join(Path(sample).parts[3:])  # Remove 'datasets/gajeshladhar/core-five/'
    url = f"https://huggingface.co/datasets/gajeshladhar/core-five/resolve/main/{filename}"
    with fsspec.open(url, 'rb') as sf:
        tr = xr.open_datatree(sf, engine='h5netcdf')
        sample_polygons.append(tr.polygon)
print(sample_polygons)
# good comparison benchmark for performance benefits of S2 matching and spatial indexing for STAC matching later

In [ ]:
# Helper functions for converting geometries to bounding boxes for SAM
# needs to be declared after xarray load so CRS is available
import s2cell
# older library that was archived in 2023 but using as workaround while duckdb extension is updated: https://github.com/sidewalklabs/s2sphere
import s2sphere as s2
from shapely.geometry import box
import geopandas as gpd

# TODO: implement required proj and transform for buffer in meters 
def s2_cell_to_geo_bbox(s2_cell_id, buffer_m=None):
    """
    Convert an S2 cell ID to bbox coordinates for SAM.
    
    Args:
        s2_cell_id: S2 cell ID (int)
        buffer_m: Buffer size in meters (default 15m for level 19)
    
    Returns:
        list: [left, bottom, right, top] in EPSG:4326
    """
    # Get S2 cell in s2sphere and convert to bounding rectangle coords
    cell_id = s2.CellId(sample_s2_cell)
    s2Cell = s2.Cell(cell_id) 
    rect_bound = s2Cell.get_rect_bound()
    return [
        rect_bound.lng_lo().degrees,  # left
        rect_bound.lat_lo().degrees,  # bottom
        rect_bound.lng_hi().degrees,  # right
        rect_bound.lat_hi().degrees   # top
    ]

# Pixel coordinate conversion functions with CRS transformation
# Copy this entire cell into your notebook to replace the existing pixel_bbox_converters cell

from affine import Affine
from pyproj import Transformer

def get_transform_from_coords(x_coords, y_coords):
    """
    Calculate affine transform from xarray coordinates.
    
    Args:
        x_coords: xarray x coordinates (in projected units, e.g., meters)
        y_coords: xarray y coordinates (in projected units, e.g., meters)
    
    Returns:
        Affine: Affine transform object
    """
    # Calculate pixel size from coordinate spacing
    pixel_width = float(x_coords[1] - x_coords[0])
    pixel_height = float(y_coords[1] - y_coords[0])
    
    # Upper-left corner (coords are pixel centers)
    x_min = float(x_coords[0]) - pixel_width / 2
    y_max = float(y_coords[0]) - pixel_height / 2
    
    # Create affine transform
    transform = Affine(pixel_width, 0.0, x_min,
                      0.0, pixel_height, y_max)
    
    print(f"✓ Transform: pixel_width={pixel_width:.6f}m, pixel_height={pixel_height:.6f}m")
    return transform

def get_chip_transform(chip_metadata):
    """
    Create a local geotransform for an image chip.
    
    Args:
        chip_metadata: Dict from display_image_chip() with 'x_coords', 'y_coords'
    
    Returns:
        Affine: Affine transform for the chip's local coordinate system
    
    Example:
        >>> chip_rgb, chip_meta = display_image_chip(tree, chip_size=512)
        >>> chip_transform = get_chip_transform(chip_meta)
        >>> bbox_pixels = geometry_to_bbox_pixels(geom, chip_transform, dst_crs=chip_meta['crs'])
    """
    return get_transform_from_coords(chip_metadata['x_coords'], chip_metadata['y_coords'])

def geo_to_pixel_coords(lon, lat, transform, src_crs='EPSG:4326', dst_crs=SAMPLE_CRS):
    """
    Convert geographic coordinates to pixel coordinates.
    
    Args:
        lon: Longitude in degrees (or x in projected units if src_crs matches image)
        lat: Latitude in degrees (or y in projected units if src_crs matches image)
        transform: Affine transform from get_transform_from_coords()
        src_crs: Source CRS (default 'EPSG:4326' for lat/lon)
        dst_crs: Destination CRS (e.g., 'EPSG:32610' for UTM 30N). If None, no transformation.
    
    Returns:
        tuple: (col, row) pixel coordinates
    """
    # Transform to image CRS if needed
    if dst_crs is not None and src_crs != dst_crs:
        transformer = Transformer.from_crs(src_crs, dst_crs, always_xy=True)
        x, y = transformer.transform(lon, lat)
    else:
        x, y = lon, lat
    
    # Convert to pixel coordinates
    inv_transform = ~transform
    col, row = inv_transform * (x, y)
    return col, row

def geometry_to_bbox_pixels(geometry, transform, src_crs='EPSG:4326', dst_crs=SAMPLE_CRS):
    """
    Convert a Shapely geometry to pixel bbox coordinates.
    
    Args:
        geometry: Shapely geometry in src_crs
        transform: Affine transform from get_transform_from_coords()
        src_crs: Source CRS of geometry (default 'EPSG:4326')
        dst_crs: Image CRS. If None, assumes geometry already in image CRS.
    
    Returns:
        tuple: (x_min, y_min, x_max, y_max) in pixel coordinates
    """
    minx, miny, maxx, maxy = geometry.bounds
    
    # Convert corners to pixel coordinates
    x_min, y_max_px = geo_to_pixel_coords(minx, maxy, transform, src_crs, dst_crs)
    x_max, y_min_px = geo_to_pixel_coords(maxx, miny, transform, src_crs, dst_crs)
    
    return (x_min, y_min_px, x_max, y_max_px)

def s2_cell_to_bbox_pixels(s2_cell_id, transform, dst_crs=SAMPLE_CRS, buffer_m=15):
    """
    Convert an S2 cell ID to pixel bbox coordinates.
    
    Args:
        s2_cell_id: S2 cell ID (int)
        transform: Affine transform from get_transform_from_coords()
        dst_crs: Image CRS. If None, no transformation.
        buffer_m: Buffer size in meters (default 15m for level 19)
    
    Returns:
        tuple: (x_min, y_min, x_max, y_max) in pixel coordinates
    """
    import s2cell
    import math
    
    # Get S2 cell center in lat/lon
    lat, lon = s2cell.cell_id_to_lat_lon(s2_cell_id)
    
    # Approximate degrees per meter at this latitude
    # TODO: convert to use duckdb geography which has function for this https://github.com/paleolimbot/duckdb-geography/blob/main/docs/function-reference.md 
    deg_per_m_lat = 1 / 111000
    deg_per_m_lon = 1 / (111000 * math.cos(math.radians(lat)))
    
    # Create geographic bbox
    half_buffer_lat = buffer_m * deg_per_m_lat
    half_buffer_lon = buffer_m * deg_per_m_lon
    
    minx = lon - half_buffer_lon
    miny = lat - half_buffer_lat
    maxx = lon + half_buffer_lon
    maxy = lat + half_buffer_lat
    
    # Convert to pixel coordinates
    x_min, y_max_px = geo_to_pixel_coords(minx, maxy, transform, 'EPSG:4326', dst_crs)
    x_max, y_min_px = geo_to_pixel_coords(maxx, miny, transform, 'EPSG:4326', dst_crs)
    
    return (x_min, y_min_px, x_max, y_max_px)

print("✓ Pixel coordinate converters ready (with CRS transformation)")
print("  - get_transform_from_coords(): Calculate transform from xarray coords")
print("  - geo_to_pixel_coords(): Convert lon/lat to pixel col/row")
print("  - geometry_to_bbox_pixels(): Convert geometry to pixel bbox")
print("  - s2_cell_to_bbox_pixels(): Convert S2 cell to pixel bbox")

### Image Display Helper - FIX for 'Invalid shape' Error

The `hr/data` group has shape `(band, y, x)` but matplotlib expects `(y, x, band)` for RGB images.

In [ ]:
# Helper function to display core-five imagery (fixes transpose issue)
import matplotlib.pyplot as plt
import numpy as np
from pyproj import Transformer

def _calculate_chip_position_from_geometry(geometry, x_coords, y_coords, chip_size, crs):
    """
    Calculate chip position to center on a geometry.
    
    Args:
        geometry: Shapely geometry in EPSG:4326
        x_coords: Full image x coordinates (projected)
        y_coords: Full image y coordinates (projected)
        chip_size: Size of chip in pixels
        crs: Image CRS (e.g., 'EPSG:32630')
    
    Returns:
        tuple: (y_start, x_start) pixel positions
    """
    # Get geometry centroid in lat/lon
    centroid = geometry.centroid
    lon, lat = centroid.x, centroid.y
    
    # Transform to image CRS
    if crs and crs != 'EPSG:4326':
        transformer = Transformer.from_crs('EPSG:4326', crs, always_xy=True)
        x_proj, y_proj = transformer.transform(lon, lat)
    else:
        x_proj, y_proj = lon, lat
    
    # Find nearest pixel indices
    x_idx = np.argmin(np.abs(x_coords - x_proj))
    y_idx = np.argmin(np.abs(y_coords - y_proj))
    
    # Calculate chip bounds centered on this point
    x_start = max(0, x_idx - chip_size // 2)
    y_start = max(0, y_idx - chip_size // 2)
    
    # Ensure chip doesn't go beyond image bounds
    if x_start + chip_size > len(x_coords):
        x_start = len(x_coords) - chip_size
    if y_start + chip_size > len(y_coords):
        y_start = len(y_coords) - chip_size
    
    # Ensure non-negative
    x_start = max(0, x_start)
    y_start = max(0, y_start)
    
    return y_start, x_start

In [ ]:
def save_chip_as_geotiff(chip_rgb, chip_metadata, output_path):
    """
    Save an image chip as a GeoTIFF with proper geospatial metadata.

    Args:
        chip_rgb: numpy array of shape (height, width, 3) with uint8 values
        chip_metadata: dict with 'x_coords', 'y_coords', 'crs' from display_image_chip
        output_path: Path to save the GeoTIFF

    Returns:
        str: Path to the saved GeoTIFF
    """
    import rasterio
    from rasterio.transform import from_bounds

    # Get chip bounds from coordinates
    x_coords = chip_metadata['x_coords']
    y_coords = chip_metadata['y_coords']
    bounds = (x_coords.min(), y_coords.min(), x_coords.max(), y_coords.max())

    # Get CRS
    crs = chip_metadata['crs']

    # Create geotransform from bounds
    height, width = chip_rgb.shape[:2]
    transform = from_bounds(*bounds, width, height)

    # Ensure uint8 dtype
    if chip_rgb.dtype != np.uint8:
        if chip_rgb.max() <= 1.0:
            chip_rgb = (chip_rgb * 255).astype(np.uint8)
        else:
            chip_rgb = chip_rgb.astype(np.uint8)

    # Save as GeoTIFF
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,
        dtype='uint8',
        crs=crs,
        transform=transform,
        compress='deflate'
    ) as dst:
        # Write RGB bands
        for i in range(3):
            dst.write(chip_rgb[:, :, i], i + 1)

    return output_path

In [ ]:
def display_image_chip(tree, chip_size=512, random_chip=True, data_type='hr', combine_rgb_bands=False, 
                       y_start=None, x_start=None, center_on_geometry=None):
    """
    Display imagery from core-five datatree (supports both HR and Sentinel-2).
    
    Args:
        tree: xarray DataTree from core-five
        chip_size: Size of chip to display (default 512x512)
        random_chip: If True, select random location; else use center (ignored if other params provided)
        data_type: 'hr' for high-res RGB or 's2' for Sentinel-2
        combine_rgb_bands: For S2, force RGB composition from B02/B03/B04
        y_start: Optional fixed y start position (overrides random_chip)
        x_start: Optional fixed x start position (overrides random_chip)
        center_on_geometry: Optional Shapely geometry to center chip on (overrides random_chip)
                           Geometry should be in EPSG:4326 (lat/lon)
    
    Returns:
        tuple: (chip_rgb, chip_metadata) where:
            - chip_rgb: numpy array of shape (chip_size, chip_size, 3)
            - chip_metadata: dict with 'y_start', 'x_start', 'x_coords', 'y_coords', 'crs'
    """

    # Pick latest time
    latest_time_idx = -1

    if data_type == 'hr':
        # HR data: shape is (band, y, x) with 3 RGB bands
        hr_data = tree['hr']['data'].values
        n_bands, height, width = hr_data.shape
        print(f"HR image shape: {hr_data.shape} (band, y, x)")
        
        # Get coordinates and CRS
        x_coords = tree['hr'].ds.x.values
        y_coords = tree['hr'].ds.y.values
        crs = tree.attrs.get('crs', None)
        
        # Select chip location
        if y_start is None or x_start is None:
            if center_on_geometry is not None:
                # Center chip on the provided geometry
                y_start, x_start = _calculate_chip_position_from_geometry(
                    center_on_geometry, x_coords, y_coords, chip_size, crs
                )
                print(f"Centering chip on geometry (centroid: {center_on_geometry.centroid.coords[0]})")
            elif random_chip:
                y_start = np.random.randint(0, max(1, height - chip_size))
                x_start = np.random.randint(0, max(1, width - chip_size))
            else:
                y_start = (height - chip_size) // 2
                x_start = (width - chip_size) // 2
        
        # Extract chip
        chip = hr_data[:, y_start:y_start+chip_size, x_start:x_start+chip_size]
        
        # Transpose from (band, y, x) to (y, x, band) for matplotlib
        chip_rgb = np.transpose(chip, (1, 2, 0))
        print(f"Chip shape after transpose: {chip_rgb.shape} (y, x, band)")
        
        # Extract chip coordinates
        chip_x_coords = x_coords[x_start:x_start+chip_size]
        chip_y_coords = y_coords[y_start:y_start+chip_size]
        
        title = f"HR RGB {chip_size}x{chip_size} chip\nLocation: y={y_start}, x={x_start}"
        
    elif data_type == 's2':
        # S2 data: shape is (time, y, x) for each band
        print(f"Using sensor with data_vars: {tree['s2'].data_vars}")
        s2_ds = tree['s2'].ds
        
        # Get coordinates and CRS
        x_coords = s2_ds.x.values
        y_coords = s2_ds.y.values
        crs = tree.attrs.get('crs', None)
        
        print(f"S2 time dimension: {len(s2_ds.time)} timestamps")
        print(f"Using latest timestamp: {s2_ds.time.values[latest_time_idx]}")
        
        # Check if 'visual' band exists (pre-computed RGB)
        if 'visual' in s2_ds.data_vars and not combine_rgb_bands:
            print("Using pre-computed 'visual' band")
            s2_data = s2_ds['visual'].isel(time=latest_time_idx).values
            height, width = s2_data.shape
            
            # Select chip location
            if y_start is None or x_start is None:
                if center_on_geometry is not None:
                    y_start, x_start = _calculate_chip_position_from_geometry(
                        center_on_geometry, x_coords, y_coords, chip_size, crs
                    )
                    print(f"Centering chip on geometry (centroid: {center_on_geometry.centroid.coords[0]})")
                elif random_chip:
                    y_start = np.random.randint(0, max(1, height - chip_size))
                    x_start = np.random.randint(0, max(1, width - chip_size))
                else:
                    y_start = (height - chip_size) // 2
                    x_start = (width - chip_size) // 2
            
            # Extract chip (single band grayscale or RGB if visual is 3-channel)
            chip = s2_data[y_start:y_start+chip_size, x_start:x_start+chip_size]
            
            # If single channel, convert to RGB by repeating
            if chip.ndim == 2:
                chip_rgb = np.stack([chip, chip, chip], axis=-1)
            else:
                chip_rgb = chip
                
        else:
            # Create RGB from B04 (Red), B03 (Green), B02 (Blue)
            print("Creating RGB from B04 (Red), B03 (Green), B02 (Blue)")
            b04 = s2_ds['B04'].isel(time=latest_time_idx).values  # Red
            b03 = s2_ds['B03'].isel(time=latest_time_idx).values  # Green
            b02 = s2_ds['B02'].isel(time=latest_time_idx).values  # Blue
            s2_data = np.stack([b04, b03, b02], axis=-1)
            
            height, width = b04.shape
            
            # Select chip location
            if y_start is None or x_start is None:
                if center_on_geometry is not None:
                    y_start, x_start = _calculate_chip_position_from_geometry(
                        center_on_geometry, x_coords, y_coords, chip_size, crs
                    )
                    print(f"Centering chip on geometry (centroid: {center_on_geometry.centroid.coords[0]})")
                elif random_chip:
                    y_start = np.random.randint(0, max(1, height - chip_size))
                    x_start = np.random.randint(0, max(1, width - chip_size))
                else:
                    y_start = (height - chip_size) // 2
                    x_start = (width - chip_size) // 2
            
            # Extract chips and stack as RGB
            r = b04[y_start:y_start+chip_size, x_start:x_start+chip_size]
            g = b03[y_start:y_start+chip_size, x_start:x_start+chip_size]
            b = b02[y_start:y_start+chip_size, x_start:x_start+chip_size]
            
            chip_rgb = np.stack([r, g, b], axis=-1)
        
        print(f"S2 image shape: {s2_data.shape if 'visual' in s2_ds.data_vars else b04.shape} (y, x)")
        print(f"Chip shape: {chip_rgb.shape} (y, x, band)")
        
        # Extract chip coordinates
        chip_x_coords = x_coords[x_start:x_start+chip_size]
        chip_y_coords = y_coords[y_start:y_start+chip_size]
        
        title = f"S2 RGB {chip_size}x{chip_size} chip\nLocation: y={y_start}, x={x_start}"
    
    elif data_type == 'landsat':

        print(f"Using sensor with data_vars: {tree['landsat'].data_vars}")
        landsat_ds = tree['landsat'].ds
        
        # Get coordinates and CRS
        x_coords = landsat_ds.x.values
        y_coords = landsat_ds.y.values
        crs = tree.attrs.get('crs', None)

        print(f"Landsat time dimension: {len(landsat_ds.time)} timestamps")
        print(f"Using latest timestamp: {landsat_ds.time.values[latest_time_idx]}")

        print("Composing RGB from Landsat red, blue, and green bands...")

        red = landsat_ds['red'].isel(time=latest_time_idx).values
        green = landsat_ds['green'].isel(time=latest_time_idx).values
        blue = landsat_ds['blue'].isel(time=latest_time_idx).values
        landsat_data = np.stack([red, green, blue], axis=-1)

        height, width = red.shape

        # Select chip location
        if y_start is None or x_start is None:
            if center_on_geometry is not None:
                y_start, x_start = _calculate_chip_position_from_geometry(
                    center_on_geometry, x_coords, y_coords, chip_size, crs
                )
                print(f"Centering chip on geometry (centroid: {center_on_geometry.centroid.coords[0]})")
            elif random_chip:
                y_start = np.random.randint(0, max(1, height - chip_size))
                x_start = np.random.randint(0, max(1, width - chip_size))
            else:
                y_start = (height - chip_size) // 2
                x_start = (width - chip_size) // 2
        
        # Extract chip
        chip_rgb = landsat_data[y_start:y_start+chip_size, x_start:x_start+chip_size]

        print(f"Landsat image shape: {landsat_data.shape} (y, x, band)")
        print(f"Chip shape: {chip_rgb.shape} (y, x, band)")
        
        # Extract chip coordinates
        chip_x_coords = x_coords[x_start:x_start+chip_size]
        chip_y_coords = y_coords[y_start:y_start+chip_size]
        
        title = f"Landsat RGB {chip_size}x{chip_size} chip\nLocation: y={y_start}, x={x_start}"
        
    else:
        raise ValueError(f"Unknown data_type: {data_type}. Use 'hr' or 's2'.")
    
    # Normalize to 0-1 range if needed
    if chip_rgb.max() > 1.0:
        chip_rgb = chip_rgb / chip_rgb.max()
    
    # only render if we're not centering on a geometry
    if center_on_geometry is None:
        # Display
        plt.figure(figsize=(10, 10))
        plt.imshow(chip_rgb)
        plt.title(title)
        plt.axis('off')
        plt.show()
    
    # Create metadata dict
    chip_metadata = {
        'y_start': y_start,
        'x_start': x_start,
        'x_coords': chip_x_coords,
        'y_coords': chip_y_coords,
        'crs': crs
    }
    
    return chip_rgb, chip_metadata

# Example usage:
# Random chip:
# chip_hr, meta = display_image_chip(tree, chip_size=512, random_chip=True, data_type='hr')
# 
# Chip centered on PV label (for bbox visualization):
# chip_hr, meta = display_image_chip(tree, chip_size=512, center_on_geometry=pv_label.geometry, data_type='hr')
# chip_transform = get_chip_transform(meta)
# bbox_pixels = geometry_to_bbox_pixels(pv_label.geometry, chip_transform, dst_crs=meta['crs'])

In [ ]:
hr_chip, hr_meta = display_image_chip(tree)

In [ ]:
# 10m resolution is ~30-40x coarser than HR, adjust chip size accordingly
s2_chip, s2_meta = display_image_chip(tree, data_type='s2', chip_size=96, combine_rgb_bands=True)

In [ ]:
# landsat's 30m gsd is 3x coarser than s2's 10m, so adjust chip size accordingly
landsat_chip, landsat_meta = display_image_chip(tree, data_type='landsat', chip_size=32)

In [ ]:
# display full tree hr image
import matplotlib.pyplot as plt
import numpy as np

def display_full_image(tree, data_type='hr', combine_rgb_bands=False, normalize=True):
    """
    Display full imagery from core-five datatree (supports both HR and Sentinel-2).
    
    Args:
        tree: xarray DataTree from core-five
        data_type: 'hr' for high-res RGB or 's2' for Sentinel-2
    
    Returns:
        image_rgb: numpy array of shape (height, width, 3)
    """

    # Pick latest time
    latest_time_idx = -1
    if data_type == 'hr':
        # HR data: shape is (band, y, x)
        hr_data = tree["hr/data"].values
        n_bands, height, width = hr_data.shape
        print(f"HR image shape: {hr_data.shape} (band, y, x)")

        # Transpose from (band, y, x) to (y, x, band) for matplotlib
        image_rgb = np.transpose(hr_data, (1, 2, 0))
        print(f"Image shape after transpose: {image_rgb.shape} (y, x, band)")
        
        title = "Full HR RGB Image"
        
    elif data_type == 's2':
        # S2 data: shape is (time, y, x) for each band
        s2_ds = tree['s2'].ds
        
        print(f"S2 time dimension: {len(s2_ds.time)} timestamps")
        print(f"Using latest timestamp: {s2_ds.time.values[latest_time_idx]}")
        
        # Check if 'visual' band exists
        if 'visual' in s2_ds.data_vars and not combine_rgb_bands:
            print("Using pre-computed 'visual' band")
            s2_data = s2_ds['visual'].isel(time=latest_time_idx).values
            
            # If single channel, convert to RGB
            if s2_data.ndim == 2:
                image_rgb = np.stack([s2_data, s2_data, s2_data], axis=-1)
            else:
                image_rgb = s2_data
        else:
            # Create RGB from B04 (Red), B03 (Green), B02 (Blue)
            print("Creating RGB from B04 (Red), B03 (Green), B02 (Blue)")
            b04 = s2_ds['B04'].isel(time=latest_time_idx).values  # Red
            b03 = s2_ds['B03'].isel(time=latest_time_idx).values  # Green
            b02 = s2_ds['B02'].isel(time=latest_time_idx).values  # Blue
            
            image_rgb = np.stack([b04, b03, b02], axis=-1)
        
        print(f"S2 image shape: {image_rgb.shape} (y, x, band)")
        title = "Full S2 RGB Image"
    
    elif data_type == 'landsat':

        landsat_ds = tree['landsat'].ds
        print(f"Landsat time dimension: {len(landsat_ds.time)} timestamps")
        print(f"Using latest timestamp: {landsat_ds.time.values[latest_time_idx]}")

        print("Composing RGB from Landsat red, blue, and green bands...")

        red = landsat_ds['red'].isel(time=latest_time_idx).values
        green = landsat_ds['green'].isel(time=latest_time_idx).values
        blue = landsat_ds['blue'].isel(time=latest_time_idx).values
        image_rgb = np.stack([red, green, blue], axis=-1)

        print(f"Landsat image shape: {image_rgb.shape} (y, x, band)")
        title = "Full Landsat RGB Image"

        # Normalize to 0-1 range if needed
        if image_rgb.max() > 1.0:
            image_rgb = image_rgb / image_rgb.max()

        print(f"Image shape: {image_rgb.shape} (y, x, band)")
        title = "Full Landsat RGB Image"
    
    else:
        raise ValueError(f"Unknown data_type: {data_type}. Use 'hr' or 's2'.")

    # Normalize to 0-1 range if needed
    if normalize or image_rgb.max() > 1.0:
        image_rgb = image_rgb / image_rgb.max()
        print("Normalized image to 0-1 range")

    # Display
    plt.figure(figsize=(15, 15))
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

    return image_rgb

In [ ]:
hr_full = display_full_image(tree, normalize=False)
# display image size in MB
print(f"Image size: {hr_full.nbytes / 1024**2:.2f} MB")

In [ ]:
s2_full = display_full_image(tree, data_type='s2', combine_rgb_bands=True, normalize=False)
print(f"Image size: {s2_full.nbytes / 1024**2:.2f} MB")

In [ ]:
landsat_full = display_full_image(tree, data_type='landsat')
print(f"Image size: {landsat_full.nbytes / 1024**2:.2f} MB")

## Step 6: Image Chipping with Bounding Boxes

**Goal**: Create a chipping function that:
1. Takes the full HR RGB image
2. Chips it to a given size in pixels
3. Uses level 19 S2 cells to draw bounding boxes
4. Saves the list of bounding boxes for SAM prompts

**Purpose**: Determine if level 19 S2 cells (~15m) are good enough for SAM box prompts,
or if we need to use actual Polygon PV labels (which would reduce matches since many labels are Points).

In [ ]:
# plot out our bbox in pixel coordinates on the hr_data image
def plot_bbox_on_image(image, bbox, color='red'):
    """
    Plot a bounding box on an image.
    
    Args:
        image: numpy array of shape (height, width, 3)
        bbox: tuple of (x_min, y_min, x_max, y_max) in pixel coordinates
        color: color of the bounding box
    """
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], linewidth=1, edgecolor=color, facecolor='none')
    plt.gca().add_patch(rect)
    plt.show()

In [ ]:
sample_pv_label = pv_gdf[pv_gdf['num_matches'] > 0].iloc[rand_sample_idx]
sample_s2_cell = sample_pv_label['s2_cell_lvl_18']
# cast to int
sample_s2_cell = int(sample_s2_cell)
# convert s2 cell to geo coords bbox
sample_s2_geom = s2_cell_to_geo_bbox(sample_s2_cell)
# get geotransform from xarray meter coords
sample_geotransform = get_transform_from_coords(tree["hr"].ds.x, tree["hr"].ds.y)
# use geotransform to convert our s2 cell bbox from lat/lon to pixel coords
# sample_s2_bbox = s2_cell_to_bbox_pixels(sample_s2_cell, sample_geotransform)
sample_s2_bbox = geometry_to_bbox_pixels(box(*sample_s2_geom), sample_geotransform)
print(f"Image shape: {hr_full.shape}")
print(f"Sample S2 cell geometry: {sample_s2_geom}")
print(f"GeoTransform for sample image: {sample_geotransform}")
print(f"Sample S2 cell bbox: {sample_s2_bbox}")
plot_bbox_on_image(hr_full, sample_s2_bbox)

In [ ]:
if sample_pv_label.geometry.geom_type == 'Polygon':
    sample_pv_bbox = geometry_to_bbox_pixels(sample_pv_label.geometry, sample_geotransform)
    print(f"Sample PV bbox: {sample_pv_bbox}")
    plot_bbox_on_image(hr_full, sample_pv_bbox, color='red')

In [ ]:
# Plot bbox on hr chip
sample_geotransform_chip = get_chip_transform(hr_meta)

# Convert PV label geometry to chip pixel coordinates
# IMPORTANT: Must pass dst_crs to transform from EPSG:4326 to image CRS!
sample_pv_bbox_chip = geometry_to_bbox_pixels(
    sample_pv_label.geometry, 
    sample_geotransform_chip,
    src_crs='EPSG:4326',
    dst_crs=hr_meta['crs']
)

# Recalculate hr_chip centered on PV label geometry
# This ensures the PV label is in the center of the chip!
hr_chip, hr_meta_centered = display_image_chip(
    tree, 
    chip_size=512, 
    center_on_geometry=sample_pv_label.geometry, 
    data_type='hr'
)

# Now get the chip transform for the CENTERED chip
sample_geotransform_chip_centered = get_chip_transform(hr_meta_centered)

# Convert PV label geometry to chip pixel coordinates using the CENTERED chip's transform
sample_pv_bbox_chip = geometry_to_bbox_pixels(
    sample_pv_label.geometry, 
    sample_geotransform_chip_centered,
    src_crs='EPSG:4326',
    dst_crs=hr_meta_centered['crs']
)

# Also convert S2 cell bbox
sample_s2_bbox_chip = s2_cell_to_bbox_pixels(
    sample_s2_cell,
    sample_geotransform_chip_centered,
    dst_crs=hr_meta_centered['crs']
)

print(f"\n=== Chip Info ===")
print(f"Chip centered on PV label: {sample_pv_label.geometry.centroid}")
print(f"Chip position in full image: y={hr_meta_centered['y_start']}, x={hr_meta_centered['x_start']}")
print(f"Chip CRS: {hr_meta_centered['crs']}")
print(f"Chip shape: {hr_chip.shape}")
print(f"\n=== Bbox Coordinates (in chip pixels) ===")
print(f"PV label bbox: {sample_pv_bbox_chip}")
print(f"S2 cell bbox: {sample_s2_bbox_chip}")

# Plot bbox on chip
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(hr_chip)

# Plot PV label bbox (red)
x_min, y_min, x_max, y_max = sample_pv_bbox_chip
rect = Rectangle(
    (x_min, y_min), 
    x_max - x_min, 
    y_max - y_min,
    linewidth=2, 
    edgecolor='red', 
    facecolor='none',
    label='PV Label'
)
ax.add_patch(rect)

# Plot S2 cell bbox (green) for comparison
x_min, y_min, x_max, y_max = sample_s2_bbox_chip
rect = Rectangle(
    (x_min, y_min),
    x_max - x_min,
    y_max - y_min,
    linewidth=2,
    edgecolor='green',
    facecolor='none',
    label='S2 Cell (lvl 18)'
)
ax.add_patch(rect)

ax.legend()
ax.set_title(f'HR Chip with Bboxes (Centered on PV Label)\nCRS: {hr_meta_centered["crs"]}')
plt.axis('off')
plt.show()

In [ ]:
# plot bbox on s2 full image
sample_geotransform_s2 = get_transform_from_coords(tree["s2"].ds.x, tree["s2"].ds.y)
sample_s2_bbox = geometry_to_bbox_pixels(box(*sample_s2_geom), sample_geotransform_s2)
print(f"Sample S2 cell bbox (S2): {sample_s2_bbox}")
plot_bbox_on_image(s2_full, sample_s2_bbox)

## Step 7: SAM Integration

**Goal**: Test SAM segmentation with our matched PV labels and imagery.

**References**:
- [Box prompts](https://samgeo.gishub.org/examples/box_prompts/)
- [SAM2 box prompts](https://samgeo.gishub.org/examples/sam2_box_prompts/)
- [Text prompts](https://samgeo.gishub.org/examples/text_prompts/)

### 7A: SAM2 with Box Prompts (Recommended)

Using level 19 S2 cells or PV polygon geometries as box prompts.

In [ ]:
# Setup
# SAM2 with Box Prompts - Using our matched data
from samgeo import SamGeo2
from samgeo.common import raster_to_vector, regularize
import numpy as np

# Sample a random PV label with matches
matched_pv = pv_gdf[pv_gdf['num_matches'] > 0]
sample_idx = np.random.randint(0, len(matched_pv))
sample_label = matched_pv.iloc[sample_idx]

print(f"Sampled PV label {sample_idx}:")
print(f"  Geometry type: {sample_label.geometry.geom_type}")
print(f"  Matched files: {sample_label['num_matches']}")
print(f"  S2 cell token (lvl {S2_CELL_LVL}): {s2cell.cell_id_to_token(int(sample_label[f's2_cell_lvl_{S2_CELL_LVL}']))}")

# Load the matched image
sample_file = sample_label['matched_files'][0]
filename = "/".join(Path(sample_file).parts[3:])
url = f"https://huggingface.co/datasets/gajeshladhar/core-five/resolve/main/{filename}"

print(f"\nLoading image: {filename}")

# Load image with fsspec
import fsspec
import xarray as xr

with fsspec.open(url, 'rb') as f:
    tree = xr.open_datatree(f, engine='h5netcdf')
    tree.load()

# Extract HR RGB chip centered on the PV label geometry
print(f"\n=== Extracting Image Chip ===")
hr_chip, hr_meta = display_image_chip(
    tree,
    chip_size=512,  # 512x512 chip
    data_type='hr',
    center_on_geometry=sample_label.geometry  # Center on PV geometry
)

print(f"Chip shape: {hr_chip.shape}")
print(f"Chip dtype: {hr_chip.dtype}")
print(f"Chip value range: [{hr_chip.min()}, {hr_chip.max()}]")
print(f"Chip CRS: {hr_meta['crs']}")

In [ ]:
# Visualize the chip with bbox overlay to verify quality
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Get chip geotransform
chip_geotransform = get_chip_transform(hr_meta)

# Convert PV label geometry to chip pixel coordinates
pv_bbox_chip = geometry_to_bbox_pixels(
    sample_label.geometry,
    chip_geotransform,
    src_crs='EPSG:4326',
    dst_crs=hr_meta['crs']
)

print(f"\n=== Bbox Coordinates (in chip pixels) ===")
print(f"PV label bbox: {pv_bbox_chip}")

# Plot bbox on chip
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(hr_chip)

# Plot PV label bbox (red)
x_min, y_min, x_max, y_max = pv_bbox_chip
rect = Rectangle(
    (x_min, y_min),
    x_max - x_min,
    y_max - y_min,
    linewidth=2,
    edgecolor='red',
    facecolor='none',
    label='PV Label Bbox'
)
ax.add_patch(rect)

ax.legend()
ax.set_title(f'HR Chip with Bbox (Centered on PV Label)\nCRS: {hr_meta["crs"]}')
plt.axis('off')
plt.show()

In [ ]:
# Create bounding boxes from geometry
# Option 1: Use PV geometry directly (if Polygon)
# Option 2: Use level 19 S2 cell
import rasterio

if sample_label.geometry.geom_type == 'Polygon':
    # Use actual PV polygon bounds
    bounds = sample_label.geometry.bounds  # (minx, miny, maxx, maxy)
    boxes = [[bounds[0], bounds[1], bounds[2], bounds[3]]]  # [left, bottom, right, top]
    print(f"\nUsing Polygon geometry as box prompt")
else:
    # Use S2 cell or buffer Point geometry
    # For now, use S2 cell bounds (approximate)
    centroid = sample_label['centroid']
    cell_size_deg = 0.0002  # ~15m at equator
    boxes = [[
        centroid.x - cell_size_deg/2,
        centroid.y - cell_size_deg/2,
        centroid.x + cell_size_deg/2,
        centroid.y + cell_size_deg/2
    ]]
    print(f"\nUsing S2 cell (level 19) as box prompt")

print(f"Box prompt (lat/lon): {boxes}")

# Save chip as GeoTIFF with proper geospatial metadata
# This is required for SAM2's point_crs parameter to work
temp_image_path = "../data/temp_sam_input.tif"
save_chip_as_geotiff(hr_chip, hr_meta, temp_image_path)

print(f"\nSaved chip as GeoTIFF to: {temp_image_path}")
print(f"  CRS: {hr_meta['crs']}")
print(f"  Chip bounds: ({hr_meta['x_coords'].min()}, {hr_meta['y_coords'].min()}, "
      f"{hr_meta['x_coords'].max()}, {hr_meta['y_coords'].max()})")

# Initialize SAM2
print(f"\nInitializing SAM2...")
sam = SamGeo2(
    model_id="sam2-hiera-small",
    automatic=False,
    device="cpu"  # Use "cuda" for GPU, "cpu" for CPU
    # Note: MPS (Apple Silicon) not supported due to float64 limitation
)

# Set image from GeoTIFF (this properly sets sam.image and sam.source)
sam.set_image(temp_image_path)
print(f"Image set successfully")
print(f"  Image shape: {sam.image.shape}")
print(f"  Image dtype: {sam.image.dtype}")

# Predict with box prompts (boxes are in EPSG:4326, SAM2 will transform to image CRS)
print(f"\nRunning SAM2 prediction...")

# Don't pass output= to avoid tensor_to_numpy bug with single boxes
masks, scores, logits = sam.predict(
    boxes=boxes,  # Boxes in EPSG:4326 (lat/lon)
    point_crs="EPSG:4326",  # SAM2 will transform from EPSG:4326 to image CRS
    multimask_output=False,
    return_results=True
)

print(f"Prediction complete!")
print(f"  Masks shape: {masks.shape}")
print(f"  Scores: {scores}")

# Save mask manually using rasterio (with geospatial metadata from input)
output_path = "../data/sam2_box_mask.tif"

# Get the mask (for single box with multimask_output=False, shape is (1, H, W))
mask = masks[0]  # Get first (and only) mask

# Save with same geospatial metadata as input
with rasterio.open(temp_image_path) as src:
    profile = src.profile.copy()
    profile.update(
        count=1,
        dtype='uint8',
        compress='deflate'
    )

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write((mask * 255).astype('uint8'), 1)

print(f"✓ Mask saved to: {output_path}")

# Convert to vector
vector_path = "../data/sam2_box_vector.geojson"
raster_to_vector(output_path, vector_path)
print(f"✓ Vector saved to: {vector_path}")

# Visualize
import matplotlib.pyplot as plt
import rasterio

# Load the mask from file
with rasterio.open(output_path) as src:
    mask = src.read(1)

plt.figure(figsize=(12, 10))
plt.imshow(hr_chip)  # Use the chipped image, not the full image
plt.imshow(mask, alpha=0.5, cmap='Reds')
plt.title(f'SAM2 Box Prompt Result\nChip size: {hr_chip.shape[0]}x{hr_chip.shape[1]}')
plt.axis('off')
plt.show()

### 7B: LangSAM with Text Prompts

Using text prompt "solar panel" to detect PV installations.

In [ ]:
# LangSAM with Text Prompts - Using our matched data
from samgeo.text_sam import LangSAM

# Use the same sampled image from above
print(f"Using same image: {filename}")
print(f"Image shape: {hr_chip.shape}")

# Initialize LangSAM
print(f"\nInitializing LangSAM...")
sam_text = LangSAM()

# Text prompt
text_prompt = "solar panel"
print(f"Text prompt: '{text_prompt}'")

# Predict
output_path_text = "../data/langsam_text_mask.tif"
print(f"\nRunning LangSAM prediction...")

sam_text.predict(
    hr_chip,
    text_prompt,
    box_threshold=0.24,  # Object detection confidence
    text_threshold=0.24,  # Text-object association confidence
    output=output_path_text
)

print(f"✓ Mask saved to: {output_path_text}")

# Convert to vector
vector_path_text = "../data/langsam_text_vector.geojson"
sam_text.raster_to_vector(output_path_text, vector_path_text)
print(f"✓ Vector saved to: {vector_path_text}")

# Visualize
sam_text.show_anns(cmap="Greens", box_color="red", blend=True)

### 7C: Original SAM with Box Prompts

Using the original Segment Anything Model (SAM) with box prompts from PV polygon bounds.

In [ ]:
# Original SAM with Box Prompts
from samgeo import SamGeo

print(f"\n=== Original SAM with Box Prompts ===")
print(f"Using same image from Step 7A")
print(f"Image shape: {hr_rgb.shape}")
print(f"Image dtype: {hr_rgb.dtype}")

# Initialize original SAM
print(f"\nInitializing SAM (vit_h)...")
sam_orig = SamGeo(
    model_type="vit_h",  # Options: vit_h, vit_l, vit_b
    automatic=False,
    sam_kwargs=None,
)

# Set image
sam_orig.set_image(hr_rgb)
print(f"Image set successfully")

# Use the same box prompts from 7A (in lat/lon)
print(f"\nBox prompts (lat/lon): {boxes}")

# Predict with box prompts
output_path_orig = "../data/sam_original_box_mask.tif"
print(f"\nRunning SAM prediction...")

sam_orig.predict(
    boxes=boxes,
    point_crs="EPSG:4326",  # Original SAM handles CRS conversion properly
    output=output_path_orig,
    dtype="uint8",
    multimask_output=False
)

print(f"✓ Mask saved to: {output_path_orig}")

# Convert to vector
vector_path_orig = "../data/sam_original_box_vector.geojson"
sam_orig.raster_to_vector(output_path_orig, vector_path_orig)
print(f"✓ Vector saved to: {vector_path_orig}")

# Visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 10))
plt.imshow(hr_rgb)
if hasattr(sam_orig, 'prediction') and sam_orig.prediction is not None:
    plt.imshow(sam_orig.prediction, alpha=0.5, cmap='Oranges')
plt.title('Original SAM Box Prompt Result')
plt.axis('off')
plt.show()

### 7D: SAM-HQ with Point Prompts

Using SAM-HQ (High Quality) with point prompts at PV label centroids.

In [ ]:
# SAM-HQ with Point Prompts
from samgeo.hq_sam import SamGeo as SamGeoHQ

print(f"\n=== SAM-HQ with Point Prompts ===")
print(f"Using same image from Step 7A")
print(f"Image shape: {hr_rgb.shape}")
print(f"Image dtype: {hr_rgb.dtype}")

# Initialize SAM-HQ
print(f"\nInitializing SAM-HQ (vit_h)...")
sam_hq = SamGeoHQ(
    model_type="vit_h",  # Options: vit_h, vit_l, vit_b, vit_tiny
    automatic=False,
    sam_kwargs=None,
)

# Set image
sam_hq.set_image(hr_rgb)
print(f"Image set successfully")

# Use centroid from the matched label as point prompt
point_coords = [[sample_label['centroid_lon'], sample_label['centroid_lat']]]
point_labels = [1]  # 1 = foreground point

print(f"\nPoint prompts (lat/lon): {point_coords}")
print(f"Point labels: {point_labels} (1=foreground)")

# Predict with point prompts
output_path_hq = "../data/sam_hq_point_mask.tif"
print(f"\nRunning SAM-HQ prediction...")

sam_hq.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    point_crs="EPSG:4326",  # SAM-HQ handles CRS conversion
    output=output_path_hq,
    dtype="uint8",
    multimask_output=False
)

print(f"✓ Mask saved to: {output_path_hq}")

# Convert to vector
vector_path_hq = "../data/sam_hq_point_vector.geojson"
sam_hq.raster_to_vector(output_path_hq, vector_path_hq)
print(f"✓ Vector saved to: {vector_path_hq}")

# Visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 10))
plt.imshow(hr_rgb)
if hasattr(sam_hq, 'prediction') and sam_hq.prediction is not None:
    plt.imshow(sam_hq.prediction, alpha=0.5, cmap='Purples')
# Mark the centroid point
from shapely.geometry import Point
centroid = Point(sample_label['centroid_lon'], sample_label['centroid_lat'])
# Convert to pixel coords for plotting
transform = get_transform_from_coords(tree["hr"].ds.x, tree["hr"].ds.y)
px, py = geo_to_pixel_coords(centroid.x, centroid.y, transform, 'EPSG:4326', tree.attrs.get('crs'))
plt.scatter(px, py, c='red', s=100, marker='*', edgecolors='white', linewidths=1.5, label='Centroid Point')
plt.title('SAM-HQ Point Prompt Result')
plt.legend()
plt.axis('off')
plt.show()

## VirtualiZarr [Future Work for processing multiple files]

#### OPTION 2A: VirtualiZarr with HTTP Store + Authentication ✅

In [ ]:

# Based on:
#   - VirtualiZarr docs: https://virtualizarr.readthedocs.io/en/stable/usage.html
#   - obstore ClientConfig: https://developmentseed.org/obstore/latest/api/store/config/
# Using HTTPStore with client_options={'default_headers': {'Authorization': 'Bearer ...'}}

from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from obstore.store import HTTPStore

# Get HuggingFace token from environment
hf_token = os.getenv('HF_TOKEN')

if not hf_token:
    print("WARNING: HF_TOKEN not found in environment.")
    print("VirtualiZarr may fail with 401 Unauthorized for private datasets.")
    print("Set HF_TOKEN in your .env file for authenticated access.")
else:
    print("✓ HF_TOKEN found - using authenticated access")

if (pv_gdf['num_matches'] > 0).sum() > 0:
    sample_file = pv_gdf[pv_gdf['num_matches'] > 0].iloc[rand_sample_idx]['matched_files'][0]
    filename = "/".join(Path(sample_file).parts[3:])
    url = f"https://huggingface.co/datasets/gajeshladhar/core-five/resolve/main/{filename}"
    
    print(f"\nVirtualizing file: {filename}")
    print(f"URL: {url}")
    
    try:
        # Create HTTP store with authentication
        # ClientConfig.default_headers accepts dict[str, str] | dict[str, bytes]
        bucket = "https://huggingface.co"
        
        # Create HTTPStore with authentication headers via client_options
        if hf_token:
            # HTTPStore with Bearer token authentication
            # This is the CORRECT way per obstore docs
            store = HTTPStore.from_url(
                bucket,
                client_options={
                    'default_headers': {
                        'Authorization': f'Bearer {hf_token}'
                    }
                }
            )
        else:
            # HTTPStore without authentication (for public datasets)
            store = HTTPStore.from_url(bucket)
        
        registry = ObjectStoreRegistry({bucket: store})
        
        # Open as virtual dataset
        print("Creating virtual dataset...")
        vds = open_virtual_dataset(
            url=url,
            registry=registry,
            parser=HDFParser(),
            loadable_variables=['time', 'x', 'y'],  # Load coords, virtualize data
            decode_times=True  # Decode CF time variables
        )
        
        print(f"\n✓ Virtual dataset created!")
        print(f"  Variables: {list(vds.data_vars)}")
        print(f"  Coordinates: {list(vds.coords)}")
        print(f"  Dimensions: {dict(vds.dims)}")
        print(f"\nMemory footprint (virtual refs only): {vds.vz.nbytes / 1024**2:.2f} MB")
        print(f"Actual data size (if loaded): {vds.nbytes / 1024**2:.2f} MB")
        
        # Show what's virtual vs loaded
        print(f"\nData variables (virtualized):")
        for var in vds.data_vars:
            print(f"  - {var}: {vds[var].shape}")
        
        print(f"\nCoordinates (loaded into memory):")
        for coord in vds.coords:
            print(f"  - {coord}: {vds[coord].shape}")
        
    except Exception as e:
        print(f"\nError: {e}")
        print(f"\nTroubleshooting:")
        print(f"1. Check if HF_TOKEN is set correctly in .env")
        print(f"2. Verify token has access to gajeshladhar/core-five dataset")
        print(f"3. Try accessing URL directly in browser to test authentication")
        import traceback
        traceback.print_exc()
else:
    print("No matches found.")

#### OPTION 2B: VirtualiZarr - Multiple Files (COMMENTED OUT - For Future E2E Pipelines)

**Note**: This approach is premature for the current notebook's depth-first workflow.

**Issue**: Fails with `ValueError: Could not find any dimension coordinates` due to:
- Different coordinate orders between `/hr` and `/s2` groups
- `spatial_ref` being a data variable instead of coordinate in `/s2`

**Future Use**: Ideal for E2E pipelines:
- Virtualize datasets → store as Icechunk/Kerchunk parquet in R2
- Materialize images at runtime with Lance or in-memory format

**Commented out for now** - see OPTION 2C below for single-group virtualization approach.

In [ ]:
# COMMENTED OUT - See markdown above for reasoning
# Uncomment for future E2E pipeline work

# from virtualizarr import open_virtual_mfdataset
# from virtualizarr.parsers import HDFParser
# from virtualizarr.registry import ObjectStoreRegistry
# from obstore.store import HTTPStore

# # Get HuggingFace token from environment
# hf_token = os.getenv('HF_TOKEN')
# 
# if not hf_token:
#     print("WARNING: HF_TOKEN not found in environment.")
#     print("Set HF_TOKEN in your .env file for authenticated access.")
# else:
#     print("✓ HF_TOKEN found - using authenticated access")
# 
# # ... rest of code commented out ...
# # See git history or uncomment for future E2E pipeline work

pass  # Placeholder to keep cell valid

#### OPTION 2C: VirtualiZarr - Single DataTree Group (Experimental)

**Exploring**: Can we virtualize individual groups from a DataTree?

**Approach**: Extract a single group (e.g., `/hr` or `/s2`) as an xarray Dataset, then virtualize it.

**Goal**: Get a manageable "feel" for loading and saving virtual datasets before scaling up.

In [ ]:
# OPTION 2C: Virtualize a single DataTree group
# Test: Can we use VirtualiZarr to virtualize a specific group from the NetCDF file?
# Approach: Use open_virtual_dataset with group parameter (if supported)

import xarray as xr
from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from obstore.store import HTTPStore

# Get HuggingFace token
hf_token = os.getenv('HF_TOKEN')

if not hf_token:
    print("WARNING: HF_TOKEN not found")
else:
    print("✓ HF_TOKEN found")

if (pv_gdf['num_matches'] > 0).sum() > 0:
    sample_file = pv_gdf[pv_gdf['num_matches'] > 0].iloc[rand_sample_idx]['matched_files'][0]
    filename = "/".join(Path(sample_file).parts[3:])
    url = f"https://huggingface.co/datasets/gajeshladhar/core-five/resolve/main/{filename}"
    
    print(f"\nFile: {filename}")
    print(f"URL: {url}")
    
    try:
        # Create HTTP store with authentication
        bucket = "https://huggingface.co"
        
        if hf_token:
            store = HTTPStore.from_url(
                bucket,
                client_options={
                    'default_headers': {
                        'Authorization': f'Bearer {hf_token}'
                    }
                }
            )
        else:
            store = HTTPStore.from_url(bucket)
        
        registry = ObjectStoreRegistry({bucket: store})
        
        # Test 1: Can we virtualize a specific HDF5 group?
        # NetCDF4 files are HDF5 files with groups like /hr/data and /s2
        print(f"\n--- Test 1: Virtualizing /hr/data group ---")
        
        # HDFParser might support group parameter
        # Let's try passing group to the parser
        try:
            vds_hr = open_virtual_dataset(
                url=url,
                registry=registry,
                parser=HDFParser(group='/hr/data'),  # Try specifying group
                loadable_variables=['x', 'y', 'band'],
                decode_times=False
            )
            
            print(f"✓ Virtualized /hr/data group!")
            print(f"  Variables: {list(vds_hr.data_vars)}")
            print(f"  Coordinates: {list(vds_hr.coords)}")
            print(f"  Dimensions: {dict(vds_hr.dims)}")
            print(f"  Virtual size: {vds_hr.vz.nbytes / 1024**2:.2f} MB")
            print(f"  Actual size: {vds_hr.nbytes / 1024**2:.2f} MB")
            
        except TypeError as e:
            if 'group' in str(e):
                print(f"✗ HDFParser doesn't support 'group' parameter")
                print(f"  Error: {e}")
                
                # Test 2: Virtualize the whole file, then extract group
                print(f"\n--- Test 2: Virtualize whole file, extract group later ---")
                
                vds_full = open_virtual_dataset(
                    url=url,
                    registry=registry,
                    parser=HDFParser(),
                    loadable_variables=['x', 'y', 'time'],
                    decode_times=True
                )
                
                print(f"✓ Virtualized full file")
                print(f"  Variables: {list(vds_full.data_vars)}")
                print(f"  Coordinates: {list(vds_full.coords)}")
                
                # Check if we can access groups from the virtual dataset
                print(f"\n  Note: VirtualiZarr returns a flat xarray.Dataset")
                print(f"  It doesn't preserve HDF5 group hierarchy")
                print(f"  All variables from all groups are flattened into one Dataset")
            else:
                raise
        
        # Conclusion
        print(f"\n--- Conclusion ---")
        print(f"VirtualiZarr flattens HDF5 groups into a single xarray.Dataset")
        print(f"For our workflow (single group at a time), direct xarray loading is simpler")
        
    except Exception as e:
        print(f"\nError: {e}")
        
        # Known issue: HDFParser doesn't support group parameter
        if "is not an HDF Group" in str(e):
            print(f"\n✗ Confirmed: HDFParser doesn't support reading specific HDF5 groups")
            print(f"  VirtualiZarr flattens all groups into a single Dataset")
            print(f"  For our workflow (one group at a time), direct xarray loading is better")
        else:
            import traceback
            traceback.print_exc()
else:
    print("No matches found.")

#### VirtualiZarr Findings

**What we tested**:
1. ✅ **Single file virtualization** (OPTION 2A) - Works with authentication
2. ⚠️ **Multiple file virtualization** (OPTION 2B) - Coordinate mismatch between groups
3. 🧪 **Single group virtualization** (OPTION 2C) - Testing if HDFParser supports group parameter

**Key findings**:
- VirtualiZarr **flattens HDF5 groups** into a single xarray.Dataset
- Doesn't preserve DataTree hierarchy (all groups merged)
- For working with **one group at a time** (our workflow), direct xarray loading is simpler

**For this notebook's depth-first workflow**:
- ✅ **Direct xarray loading** is the right approach
- ✅ Loads individual DataTree groups on-demand
- ✅ Simple, straightforward, works perfectly for SAM segmentation

**VirtualiZarr is better suited for**:
- 📦 E2E pipelines virtualizing thousands of files
- 💾 Storing references in Icechunk/Kerchunk for repeated access
- 🚀 Materializing data at runtime from virtual stores
- 🔄 When you need to work with the **entire file** (all groups), not individual groups

**Next**: Continue with image display and SAM segmentation workflow using direct xarray loading.